# Composite Pattern

## Intent
Compose objects into tree structures to represent part-whole hierarchies. Composite lets clients treat individual objects and compositions of objects uniformly.

## Problem
You need to work with tree structures where:
- Individual objects and groups should be treated the same
- You want to represent part-whole hierarchies
- Operations should work on both leaves and composites

**Real-world examples:**
- File systems (files and folders)
- UI components (buttons and containers)
- Organization charts
- Menu systems

## When to Use
✅ **Use when:**
- Implementing tree structures
- Want uniform treatment of objects and composites
- Part-whole hierarchies are needed
- Simplify client code with recursive structures

❌ **Avoid when:**
- Objects don't form tree structure
- Different operations for leaves vs composites
- Simple flat structure suffices

## Pattern Structure
```
        ┌───────────┐
        │ Component │
        ├───────────┤
        │ operation()│
        └───────────┘
              ▲
      ┌───────┴───────┐
  ┌───┴───┐     ┌─────┴──────┐
  │  Leaf │     │  Composite │
  └───────┘     ├────────────┤
                │ children   │
                │ add()      │
                │ remove()   │
                └────────────┘
```

## Example 1: File System (Without Composite)

**Problem**: Different handling for files and folders

In [ ]:
# WITHOUT Composite - Manual handling of files vs folders
class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size

class Folder:
    def __init__(self, name):
        self.name = name
        self.items = []

# Client has to check types!
def calculate_size(item):
    if isinstance(item, File):
        return item.size
    elif isinstance(item, Folder):
        total = 0
        for sub_item in item.items:
            total += calculate_size(sub_item)  # Recursion
        return total
    # Adding new types requires modifying this function!

## Implementation: Composite Pattern

In [ ]:
from abc import ABC, abstractmethod
from typing import List

# Component interface
class FileSystemComponent(ABC):
    """Abstract component for file system."""
    
    def __init__(self, name: str):
        self.name = name
    
    @abstractmethod
    def get_size(self) -> int:
        """Get size in bytes."""
        pass
    
    @abstractmethod
    def display(self, indent: str = ""):
        """Display structure."""
        pass


# Leaf (individual object)
class File(FileSystemComponent):
    """Leaf node - represents a file."""
    
    def __init__(self, name: str, size: int):
        super().__init__(name)
        self.size = size
    
    def get_size(self) -> int:
        return self.size
    
    def display(self, indent: str = ""):
        print(f"{indent}📄 {self.name} ({self.size} bytes)")


# Composite (container object)
class Folder(FileSystemComponent):
    """Composite node - represents a folder containing files/folders."""
    
    def __init__(self, name: str):
        super().__init__(name)
        self.children: List[FileSystemComponent] = []
    
    def add(self, component: FileSystemComponent):
        """Add a file or folder."""
        self.children.append(component)
    
    def remove(self, component: FileSystemComponent):
        """Remove a file or folder."""
        self.children.remove(component)
    
    def get_size(self) -> int:
        """Recursively calculate total size."""
        return sum(child.get_size() for child in self.children)
    
    def display(self, indent: str = ""):
        """Display folder structure."""
        print(f"{indent}📁 {self.name}/")
        for child in self.children:
            child.display(indent + "  ")


# Demo
print("=== File System Structure ===")

# Create structure
root = Folder("root")

# Add files to root
root.add(File("readme.txt", 100))
root.add(File("config.json", 250))

# Create documents folder
documents = Folder("documents")
documents.add(File("report.pdf", 5000))
documents.add(File("notes.txt", 300))

# Create photos folder
photos = Folder("photos")
photos.add(File("vacation.jpg", 15000))
photos.add(File("family.jpg", 12000))

# Create nested structure
work = Folder("work")
work.add(File("project.docx", 8000))
documents.add(work)  # Nested folder!

# Add folders to root
root.add(documents)
root.add(photos)

# Display structure
root.display()

# Calculate sizes (works uniformly!)
print(f"\nTotal size: {root.get_size()} bytes")
print(f"Documents size: {documents.get_size()} bytes")
print(f"Photos size: {photos.get_size()} bytes")

## Real-World Example: Organization Chart

In [ ]:
# Component
class Employee(ABC):
    """Abstract employee component."""
    
    def __init__(self, name: str, position: str, salary: float):
        self.name = name
        self.position = position
        self.salary = salary
    
    @abstractmethod
    def get_salary_cost(self) -> float:
        pass
    
    @abstractmethod
    def display(self, indent: str = ""):
        pass


# Leaf
class IndividualContributor(Employee):
    """Leaf - employee with no reports."""
    
    def get_salary_cost(self) -> float:
        return self.salary
    
    def display(self, indent: str = ""):
        print(f"{indent}👤 {self.name} - {self.position} (${self.salary:,.0f})")


# Composite
class Manager(Employee):
    """Composite - employee with reports."""
    
    def __init__(self, name: str, position: str, salary: float):
        super().__init__(name, position, salary)
        self.reports: List[Employee] = []
    
    def add_report(self, employee: Employee):
        self.reports.append(employee)
    
    def remove_report(self, employee: Employee):
        self.reports.remove(employee)
    
    def get_salary_cost(self) -> float:
        """Total cost including all reports."""
        total = self.salary
        for report in self.reports:
            total += report.get_salary_cost()
        return total
    
    def display(self, indent: str = ""):
        print(f"{indent}👔 {self.name} - {self.position} (${self.salary:,.0f})")
        for report in self.reports:
            report.display(indent + "  ")


# Demo
print("\n=== Organization Chart ===")

# Create organization structure
ceo = Manager("Alice", "CEO", 200000)

# Engineering department
vp_eng = Manager("Bob", "VP Engineering", 150000)
eng_mgr1 = Manager("Charlie", "Engineering Manager", 120000)
eng_mgr1.add_report(IndividualContributor("Dave", "Senior Engineer", 100000))
eng_mgr1.add_report(IndividualContributor("Eve", "Engineer", 90000))
vp_eng.add_report(eng_mgr1)
vp_eng.add_report(IndividualContributor("Frank", "Architect", 130000))

# Sales department
vp_sales = Manager("Grace", "VP Sales", 140000)
vp_sales.add_report(IndividualContributor("Henry", "Account Manager", 80000))
vp_sales.add_report(IndividualContributor("Iris", "Sales Rep", 70000))

# Add to CEO
ceo.add_report(vp_eng)
ceo.add_report(vp_sales)
ceo.add_report(IndividualContributor("Jack", "Executive Assistant", 60000))

# Display org chart
ceo.display()

# Calculate costs
print(f"\n💰 Total company salary cost: ${ceo.get_salary_cost():,.0f}")
print(f"💰 Engineering cost: ${vp_eng.get_salary_cost():,.0f}")
print(f"💰 Sales cost: ${vp_sales.get_salary_cost():,.0f}")

## Real-World Example: UI Component Tree

In [ ]:
# Component
class UIComponent(ABC):
    """Abstract UI component."""
    
    def __init__(self, name: str):
        self.name = name
    
    @abstractmethod
    def render(self, indent: str = ""):
        pass


# Leaf components
class Button(UIComponent):
    """Leaf - button component."""
    
    def __init__(self, name: str, label: str):
        super().__init__(name)
        self.label = label
    
    def render(self, indent: str = ""):
        print(f"{indent}🔘 Button: {self.label}")


class TextBox(UIComponent):
    """Leaf - text input component."""
    
    def __init__(self, name: str, placeholder: str):
        super().__init__(name)
        self.placeholder = placeholder
    
    def render(self, indent: str = ""):
        print(f"{indent}📝 TextBox: {self.placeholder}")


class Label(UIComponent):
    """Leaf - label component."""
    
    def __init__(self, name: str, text: str):
        super().__init__(name)
        self.text = text
    
    def render(self, indent: str = ""):
        print(f"{indent}🏷️  Label: {self.text}")


# Composite components
class Panel(UIComponent):
    """Composite - container for components."""
    
    def __init__(self, name: str):
        super().__init__(name)
        self.children: List[UIComponent] = []
    
    def add(self, component: UIComponent):
        self.children.append(component)
    
    def remove(self, component: UIComponent):
        self.children.remove(component)
    
    def render(self, indent: str = ""):
        print(f"{indent}📦 Panel: {self.name}")
        for child in self.children:
            child.render(indent + "  ")


class Form(UIComponent):
    """Composite - form container."""
    
    def __init__(self, name: str, title: str):
        super().__init__(name)
        self.title = title
        self.children: List[UIComponent] = []
    
    def add(self, component: UIComponent):
        self.children.append(component)
    
    def render(self, indent: str = ""):
        print(f"{indent}📋 Form: {self.title}")
        for child in self.children:
            child.render(indent + "  ")


# Demo
print("\n=== UI Component Tree ===")

# Create login form
login_form = Form("login", "Login Form")

# Add username field
username_panel = Panel("username_panel")
username_panel.add(Label("username_label", "Username:"))
username_panel.add(TextBox("username_input", "Enter username"))

# Add password field
password_panel = Panel("password_panel")
password_panel.add(Label("password_label", "Password:"))
password_panel.add(TextBox("password_input", "Enter password"))

# Add button panel
button_panel = Panel("button_panel")
button_panel.add(Button("submit", "Login"))
button_panel.add(Button("cancel", "Cancel"))

# Assemble form
login_form.add(username_panel)
login_form.add(password_panel)
login_form.add(button_panel)

# Render the entire UI tree
login_form.render()

print("\n✅ Uniform treatment: all components have render() method!")

## Transparent vs Safe Composite

### Transparent Composite
Leaf and Composite have same interface (child management in Component).

In [ ]:
# Transparent - all have add/remove (may not make sense for leaves)
class TransparentComponent(ABC):
    @abstractmethod
    def operation(self):
        pass
    
    # These are in base class
    def add(self, component):
        raise NotImplementedError("Cannot add to leaf")
    
    def remove(self, component):
        raise NotImplementedError("Cannot remove from leaf")

### Safe Composite
Only Composite has child management methods (type-safe).

In [ ]:
# Safe - only composite has add/remove (what we used above)
class SafeComponent(ABC):
    @abstractmethod
    def operation(self):
        pass
    # No add/remove here

class SafeComposite(SafeComponent):
    def __init__(self):
        self.children = []
    
    def add(self, component):  # Only in composite
        self.children.append(component)
    
    def remove(self, component):
        self.children.remove(component)
    
    def operation(self):
        for child in self.children:
            child.operation()

## Advantages & Disadvantages

### ✅ Advantages
1. **Uniform treatment**: Same interface for leaves and composites
2. **Easy to add new components**: Open/Closed Principle
3. **Simplifies client code**: No type checking needed
4. **Recursive structure**: Natural for tree operations
5. **Flexible**: Easy to build complex hierarchies

### ❌ Disadvantages
1. **Overly general**: May be hard to restrict component types
2. **Difficult type constraints**: All components must share interface
3. **Safety vs transparency**: Trade-off in design

## Common Use Cases

1. **File Systems**: Files and directories
2. **UI Frameworks**: Widgets and containers
3. **Graphics**: Shapes and groups
4. **Menus**: Menu items and submenus
5. **Organizations**: Employees and departments
6. **DOM Tree**: HTML elements

## Related Patterns

- **Decorator**: Similar structure, different intent (adds behavior)
- **Iterator**: Often used to traverse composite structures
- **Visitor**: Perform operations on composite structures
- **Chain of Responsibility**: Can use composite for handling

## Best Practices

1. **Choose transparency**: Decide between safe vs transparent composite
2. **Cache results**: For expensive operations (like size calculation)
3. **Parent references**: Consider adding parent pointer for navigation
4. **Ordering**: Decide if child order matters
5. **Shared components**: Be careful with components in multiple composites

## Summary

Composite pattern enables:
- Tree structure representation
- Uniform treatment of objects
- Recursive composition
- Part-whole hierarchies

Perfect for: File systems, UI components, organization charts, graphics, menus.

**Key Insight**: Treat individual objects and compositions uniformly through common interface!